In [7]:
import json, codecs, os
root = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/'

In [8]:
nodes_fpath = os.path.join(root, 'text2cypher/raw_graph_schema/node_properties.json')
rels_fpath = os.path.join(root, 'text2cypher/raw_graph_schema/rel_properties.json')

In [9]:
sorted_schema = {} # {'a':{'b':[], 'c':[]}}
for fpath in [nodes_fpath, rels_fpath]:
    with codecs.open(fpath, 'r', 'utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if 'nodeType' in every:
                tag = every['nodeType']
            elif 'relType' in every:
                tag = every['relType']

            if tag not in sorted_schema:
                sorted_schema[tag]= {}
                sorted_schema[tag]['property_names']=[]
                sorted_schema[tag]['property_types']=[]

            sorted_schema[tag]['property_names'].append(every['propertyName'])
            if isinstance(every['propertyTypes'], list) and len(every['propertyTypes'])==1:
                sorted_schema[tag]['property_types'].append('_'.join(every['propertyTypes']))
            else:
                sorted_schema[tag]['property_types'].append(every['propertyTypes'])

In [10]:
sorted_schema


{':`flight_company.airport`': {'property_names': ['name',
   'IATA',
   'Country',
   'ICAO',
   'id',
   'City'],
  'property_types': ['String',
   'String',
   'String',
   'String',
   'Long',
   'String']},
 ':`flight_company.operate_company`': {'property_names': ['name',
   'id',
   'Type',
   'Principal_activities',
   'Incorporated_in',
   'Group_Equity_Shareholding'],
  'property_types': ['String',
   'Long',
   'String',
   'String',
   'String',
   'Double']},
 ':`flight_company.flight`': {'property_names': ['id',
   'Pilot',
   'airport_id',
   'company_id',
   'Vehicle_Flight_number',
   'Velocity',
   'Date',
   'Altitude'],
  'property_types': ['Long',
   'String',
   'Long',
   'Long',
   'String',
   'Double',
   'String',
   'Double']},
 ':`employee_hire_evaluation.employee`': {'property_names': ['City',
   'Employee_ID',
   'Age',
   'Name'],
  'property_types': ['String', 'Long', 'Long', 'String']},
 ':`employee_hire_evaluation.shop`': {'property_names': ['Name',
   

In [11]:
import os
outfpath=os.path.join(root, 'text2cypher/schema.json')
with open(outfpath, 'w') as out:
    json.dump(sorted_schema, out, indent=4)

In [13]:
for split in ['train.json', 'dev.json']:
    fpth = os.path.join(root, 'spider', split )
    print(fpth)
    reformated_data = []
    with open(fpth, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            print(every)
            reformated = {}
            reformated['question'] = every['question']
            reformated['query'] = every['cypher_query']
            reformated['db_id'] = every['db_id']
            reformated['answers'] = every['answers']
            reformated_data.append(reformated)
        foutfpath=os.path.join(root, 'text2cypher', split)
        with open(foutfpath, 'w') as out:
            json.dump(reformated_data, out, indent=4)

                    


/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/spider/train.json
{'db_id': 'department_management', 'cypher_query': 'MATCH (head:`department_management.head`)\nWHERE head.age > 56\nRETURN count(*)', 'parsed_cypher': {'Token_Keyword': ['MATCH', 'WHERE', 'RETURN'], 'Token_Text_Whitespace': [' ', '\n'], 'Token_Punctuation': ['(', ':', ')'], 'Token_Name_Variable': ['head', 'age'], 'Token_Name_Label': '`department_management.head`', 'Token_Operator': ['.', '>', '*'], 'Token_Literal_Number_Integer': '56', 'Token_Name_Function': 'count'}, 'question': 'How many heads of the departments are older than 56 ?', 'answers': [[5]]}
{'db_id': 'department_management', 'cypher_query': 'MATCH (head:`department_management.head`)\nRETURN head.name,head.born_state,head.age\nORDER BY head.age', 'parsed_cypher': {'Token_Keyword': ['MATCH', 'RETURN', 'name', 'ORDER BY'], 'Token_Text_Whitespace': [' ', '\n'], 'Token_Punctuation': ['(', ':', ')', ','], 'Token_Name_Variable': ['head', 'born_